# ÁLGEBRA LINEAR COMPUTACIONAL

## SEGUNDA AVALIAÇÃO - 2020.2 - PROFESSOR ERITO MARQUES   

---
### GRUPO:
**Ícaro Santos Barcelos Pereira - _2017780187_**\
**Leandro Bataglia Pereira - _2018780199_**\
**Pedro Henrique Farolfi Camelo - _2018780326_**

---

Repositório dos algoritmos solicitados na segunda prova da disciplina de Álgebra Linear Computacional.
##### Instruções:
As respostas devem ser enviadas no formato jupyter notebook.\
Os trabalhos devem ser feitos em grupo. Não é permitida troca de informações entre os grupos.\
Fixada a data de entrega, não serão aceitos trabalhos entregues fora dessa data.

In [76]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore",category=UserWarning)

In [77]:
import numpy as np
import pandas as pd
import math
from normas import *
from criterios import *
from cholesky import *
from iterativos import *
from benford import *
from fatoracaoLU import fatoracaoLU
from substituicao import resolve_substituicao
from sympy import Matrix, init_printing
init_printing()
#BIBLIOTECA QUE NÃO ESTOU USANDO MAS PODE SER ÚTIL PARA EXIBIR AS MATRIZES/VETORES = display(Matrix(*variavel que representa o dado*)

In [78]:
# Coverti isso aqui em um procedimento, a parada é que vc pode mandar isso
# ocorrer no início de qualquer célula e pode alterar o excel em tempo de execução :)

# CONVERSÃO DOS DADOS EM ARRAYS DA NUMPY COM TYPE 'float64'
def ler_matriz_A_vetor_b(file_excel, sheet_matriz, sheet_vetor):
    df_matriz = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name=sheet_matriz) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name=sheet_vetor) # Le o arquivo do excel que contém o vetor b sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor = np.ravel(np.array(df_vetor, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor

### 1) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e avalie:

In [79]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

**a)** se A é tridiagonal;

In [80]:
def verifica_tridiagonal(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> não é tridiagonal")
        return False

    for i in range(linhas):
        for j in range(colunas):
            if np.abs(i-j) <= 1 and matriz_A[i,j] == 0:
                return False
            if np.abs(i-j) >= 2 and matriz_A[i,j] != 0:
                return False
    return True

In [81]:
verifica_tridiagonal(matriz, debug=True)

False

In [82]:
#declarando vetor na mão:
A = np.array([[1,1,0],
              [1,1,1],
              [0,1,1]])
verifica_tridiagonal(A)

True

**b)** se A é triangular superior;

In [83]:
def verifica_tri_superior(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> não é triangular superior")
        return False

    for i in range(linhas):
        for j in range(i):
            if matriz_A[i,j] != 0:
                return False
    return True

In [84]:
verifica_tri_superior(matriz, debug=True)

False

**c)** se A é triangular inferior;

In [85]:
def verifica_tri_inferior(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        print("A matriz não é quadrada -> não é triangular inferior")
        return False

    for j in range(colunas):
        for i in range(j):
            if matriz_A[i,j] != 0:
                return False
    return True

In [86]:
verifica_tri_inferior(matriz)

False

**d)** se Traço de A é maior que 4.

In [87]:
def traco(matriz_A, debug=False):
    linhas, colunas = np.shape(matriz_A)
    
    try:
        assert linhas == colunas
    except:
        if debug:
            print("A matriz não é quadrada -> traço não está definido.")
        return False
    
    n, traco = min((linhas, colunas)), 0
    for i in range(n):
        traco += matriz_A[i,i]
    if traco > 4:
        print("O traço da matriz é " + str(traco) + ", portanto é maior que 4")
        return True
    print("O traço da matriz é " + str(traco) + ", portanto não é maior que 4")
    return False

In [88]:
A = np.array([[1,2,3],[1,2,3]])
traco(A, debug=True)

A matriz não é quadrada -> traço não está definido.


False

In [89]:
traco(matriz, debug=True)

O traço da matriz é 1179.0, portanto é maior que 4


True

---
### 2) Implemente um algoritmo que leia uma matriz A (triangular) e um vetor b (de uma planilha excel) e resolva o sistema Ax = b por substituição para frente ou para trás conforme o caso.

In [90]:
def resolve_substituicao(matriz_A, vetor_b):
    
    if checa_tamanho_matriz_vetor(matriz_A, vetor_b) == False:
        return False
        
    if(verifica_tri_inferior(matriz_A)):
        return resolve_subs_frente(matriz_A, vetor_b)

    elif(verifica_tri_superior(matriz_A)):
        return resolve_subs_tras(matriz_A,vetor_b)
        
    else:
        print("A matriz não é triangular")
        return False
        
        
def checa_tamanho_matriz_vetor(matriz_A, vetor_b):
    lin_A, col_A = np.shape(matriz_A)
    col_b = np.shape(vetor_b)
    
    try:
        assert lin_A == col_b[0]
    except:
        print("A quantidade de linhas em A e de colunas em b são distintas")
        return False
    return True
    
    
def resolve_subs_frente(matriz_A, vetor_b): # Resolve um sistema com matriz triangular inferior
    n = len(matriz_A)
    vetor_resultado = np.array(np.copy(vetor_b), dtype='f8')

    for i in range(n):
        for j in range(i):
            vetor_resultado[i] = vetor_resultado[i] - matriz_A[i,j] * vetor_resultado[j]
        vetor_resultado[i] = vetor_resultado[i]/matriz_A[i,i]
    return vetor_resultado

def resolve_subs_tras(matriz_A,vetor_b): # Resolve um sistema com matriz triangular superior
    n = len(matriz_A)
    vetor_resultado = np.array(np.copy(vetor_b), dtype='f8')

    for i in range(n-1,-1,-1):
        for j in range(n-1,i,-1):
            vetor_resultado[i] = vetor_resultado[i] - matriz_A[i,j] * vetor_resultado[j]
        vetor_resultado[i] = vetor_resultado[i]/matriz_A[i,i]
    return vetor_resultado

In [91]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

delta_x = resolve_substituicao(matriz, vetor)

if delta_x != False:
    print("Vetor x resultado da substituição:\n",delta_x,"\n")

A matriz não é triangular


---
### 3) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:



In [92]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

**a)** Avalie se é possível obter o fator de Cholesky;

In [93]:
def checa_positiva_definida(matriz_A, debug=False):
        
    n = len(matriz_A) # n é a ordem da matriz a
    matriz_R = np.copy(np.triu(matriz_A)) # matriz_r é uma cópia triangular superior da matriz A
    for i in range(n):
            for k in range(i): # Não é executado quando i = 0
                matriz_R[i,i] = matriz_R[i,i] - math.pow(matriz_R[k,i],2) # Define os termos da diagonal principal
            if matriz_R[i,i] <= 0: # Se algum elemento da diagonal principal for menor ou igual a zero, por definição, a matriz não é positiva definida
                if debug:
                    print("Erro: A matriz não é positiva definida.")
                return False
            matriz_R[i,i] = math.sqrt(matriz_R[i,i]) 
            for j in range(i+1,n): # Não é executado quando i = n
                for k in range(i): # Não é executado quando i = 0
                    matriz_R[i,j] = matriz_R[i,j] - (matriz_R[k,i] * matriz_R[k,j])
                matriz_R[i,j] = matriz_R[i,j]/matriz_R[i,i]
    return True         

In [94]:
A = np.array([[2.,1.,0.],
              [1.,2.,-1.],
              [0.,-1.,2.]])

In [95]:
A = np.array([[2.,1.,0.],
              [1.,2.,-1.],
              [0.,-1.,2.]])

In [96]:
b = np.array([[3.],[0.],[1.]])

In [97]:
checa_positiva_definida(A, debug=True)

True

**b)** Resolva o sistema Ax = b usando a fatoração de Cholesky.

In [98]:
def decomposicao_cholesky(matriz_A, vetor_b):
    n = len(matriz_A) # n é a ordem da matriz a
    matriz_R = np.copy(np.triu(matriz_A)) # matriz_r é uma cópia triangular superior da matriz A

    for i in range(n):
        for k in range(i): # Não é executado quando i = 0
            matriz_R[i,i] = matriz_R[i,i] - math.pow(matriz_R[k,i],2) # Define os termos da diagonal principal
        if matriz_R[i,i] <= 0: # Se algum elemento da diagonal principal for menor ou igual a zero, por definição, a matriz não é positiva definida
            print("Erro: A matriz não é positiva definida.")
            return
        matriz_R[i,i] = math.sqrt(matriz_R[i,i]) 
        for j in range(i+1,n): # Não é executado quando i = n
            for k in range(i): # Não é executado quando i = 0
                matriz_R[i,j] = matriz_R[i,j] - (matriz_R[k,i] * matriz_R[k,j])
            matriz_R[i,j] = matriz_R[i,j]/matriz_R[i,i]

    return resolve_cholesky(matriz_R, vetor_b)

def resolve_cholesky(matriz_R, vetor_b):
    matriz_Rt = np.copy(np.transpose(matriz_R)) # matriz_rt é a matriz R transposta
    
    vetor_y = subs.resolve_substituicao(matriz_Rt, vetor_b) # Faz o cálculo de Rt * y = b para obter o vetor y
    vetor_x = subs.resolve_substituicao(matriz_R, vetor_y) # Faz o cálculo de R * x = y para obter o vetor x (resultado)

    return vetor_x # Retorna o vetor x (resultado)

In [99]:
# TESTE DO ALGORITMO DE DECOMPOSIÇÃO DE CHOLESKY E NORMA RESIDUAL (cholesky.py, normas.py)
delta_x = decomposicao_cholesky(matriz, vetor)
print("Vetor x resultado da Decomposição de Cholesky:\n",delta_x,"\n")
#residual(matriz, vetor, delta_x)

Erro: A matriz não é positiva definida.
Vetor x resultado da Decomposição de Cholesky:
 None 



---
### 4) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:

In [100]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

**a)** Avalie se algum critério é satisfeito para aplicação dos métodos de Gauss-Seidel, Jacobi e SOR;

In [101]:
def criterio_linhas(matriz_A): # Verificar o critério das linhas para o método de Jacobi
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    soma = 0

    for i in range(n): # Percorre toda a matriz
        for j in range(m):
            if i != j: # Desconsidera os termos da diagonal principal
                soma += np.abs(matriz_A[i,j])

        if soma >= np.abs(matriz_A[i,i]): # Verifica se a soma de todos os termos (sem contar o termo da diagonal principal) de cada linha é menor ou maior que o termo da diagonal principal (Critério das linhas)
            return False

        soma = 0
    return True

In [102]:
def criterio_sassenfeld(matriz_A):
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que auxilia nos somatórios dos termos tanto de Aij*Beta, quanto de Aij
    vetor_betao = np.zeros(m) # Vetor de Betas iniciado com termos iguais a zero

    for i in range(n):
        for j in range(i): # j vai até i-1 (desconsidera a primeira iteração)
            aux += np.abs(matriz_A[i,j]) * vetor_betao[j] # A partir da segunda iteração, inclui a multiplicação de Aij pelos Betas anteriores no somatório (já que na primeira iteração não há Betas anteriores, obviamente)
        for j in range(i,n):
            aux += np.abs(matriz_A[i,j]) # Somatório dos módulos dos termos de Aij
        vetor_betao[i] = aux/np.abs(matriz_A[i,i]) #Divide o valor da soma pelo termo da diagonal principal
        aux = 0 # Zera a variável auxiliar pra próxima iteração
    
    print("Maior betão:", np.max(vetor_betao))
    if np.max(vetor_betao) < 1: # Verifica se a condição do critério de Sassenfeld (max(B) < 1) é satisfeita
        return True
    return False

In [103]:
criterio_linhas(matriz)

False

In [104]:
criterio_sassenfeld(matriz)

Maior betão: 10602274533433.357


False

**b)** Resolva o sistema Ax = b usando os métodos iterativos de Gauss-Seidel, Jacobi e SOR.

## Jacobi

In [105]:
def jacobi(matriz_A, vetor_b, vetor_x=0, N=10000, p=1e-15): # Função que executa o algoritmo de Jacobi
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj

    if isinstance(vetor_x, int): # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)
    
    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1

    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função (N) se a condição de parada não for satisfeita
        for i in range(n):
            for j in range(n):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj
            
            vetor_aux[i] = (vetor_b[i] - aux)/matriz_A[i,i] # Executa a iteração do método com (bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        termos_encontrados = 0
        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0:
                if np.abs(vetor_aux[i] - vetor_x[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p 
                    termos_encontrados += 1
            else:
                if np.abs(vetor_aux[i] - vetor_x[i]) <= p*np.abs(vetor_x[i]):
                    termos_encontrados += 1
                    
        if(termos_encontrados == n):
            print("(Jacobi) Resultado encontrado com o critério de parada, na iteração:", k)
            return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(Jacobi) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [106]:
print(jacobi(matriz, vetor, N=100))

ATENÇÃO: O critério das linhas não foi satisfeito.
(Jacobi) Resultado encontrado com o número máximo de iterações.
[-2.21507407e+191 -7.85062201e+191 -4.02063148e+191 -3.00124088e+191
 -2.34581480e+192 -2.32464880e+191 -4.58103429e+192 -4.79832453e+191
 -5.85375866e+191 -4.90112952e+191 -2.01091315e+191 -2.59706719e+191
 -6.26779852e+191 -1.21273532e+192 -8.04504337e+192 -2.42684232e+192
 -4.65327284e+191 -2.98587023e+191 -3.29553950e+191 -4.17899256e+191
 -4.88807065e+191 -4.20548399e+191 -3.16760574e+192 -1.26588784e+192
 -1.08434271e+193]


## Gauss-Seidel

In [107]:
def seidel(matriz_A, vetor_b, vetor_x=0, N=1000, p=1e-15): # Função que executa o algoritmo de Gauss-Seidel
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj

    if isinstance(vetor_x, int): # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)

    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1
    	
    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    if criterio_sassenfeld(matriz_A): # Verifica se o critério de Sassenfeld é satisfeito
        print("O critério de Sassenfeld foi satisfeito.")
    else:
        print("ATENÇÃO: O critério de Sassenfeld não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função
        for i in range(n):
            for j in range(m):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    if i > j:
                        aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
                    else:
                        aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x

            vetor_aux[i] = (vetor_b[i] - aux)/matriz_A[i,i] # Executa a iteração do método com (bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        termos_encontrados = 0
        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0:
                if np.abs(vetor_aux[i] - vetor_x[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p 
                    termos_encontrados += 1
            else:
                if np.abs(vetor_aux[i] - vetor_x[i]) <= p*np.abs(vetor_x[i]):
                    termos_encontrados += 1
                    
        if(termos_encontrados == n):
                print("(Seidel) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(Seidel) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [108]:
print(seidel(matriz, vetor))

ATENÇÃO: O critério das linhas não foi satisfeito.
Maior betão: 10602274533433.357
ATENÇÃO: O critério de Sassenfeld não foi satisfeito.


<ipython-input-107-2310ea6e5e6a>:25: RuntimeWarning: overflow encountered in double_scalars
  aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
<ipython-input-107-2310ea6e5e6a>:25: RuntimeWarning: invalid value encountered in double_scalars
  aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
<ipython-input-107-2310ea6e5e6a>:27: RuntimeWarning: invalid value encountered in double_scalars
  aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x
<ipython-input-107-2310ea6e5e6a>:27: RuntimeWarning: overflow encountered in double_scalars
  aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não fo

(Seidel) Resultado encontrado com o número máximo de iterações.
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]


## SOR

In [109]:
def calcular_omega_otimo(matriz_A):
    
    if checa_positiva_definida(matriz_A, debug=True) == False or verifica_tridiagonal(matriz_A, debug=True) == False:
        print("Não há como encontrar w ótimo.")
        return None        
    
    D = np.copy(matriz_A)
    for i in range(3):
        for j in range(3):
            if i != j:
                D[i,j] = 0.
    D = np.linalg.inv(D)
    
    L = np.tril(A)
    U = np.triu(A)

    for i in range(3):
        for j in range(3):
            if i == j:
                L[i,j] = 0.
                U[i,j] = 0.

    LU = L + U
    T = np.matmul(D, LU)
    P = np.linalg.eig(T)
    p = np.amax(P[0])
    
    print("omega: {}".format(2/(1 + math.sqrt(1 - pow(p, 2)))))
    return 2/(1 + math.sqrt(1 - pow(p, 2)))
    

In [110]:
def sor(matriz_A, vetor_b, vetor_x=0, N=10000, p=1e-15, w=1.7): # Quando w = 1 é igual ao Seidel
    n, m = len(matriz_A), len(matriz_A[0]) # n e m são as dimensões da matriz
    aux = 0 # Variável que vai auxiliar a armazenar as subtrações dos valores Aij*xj
    
    omega = calcular_omega_otimo(matriz_A)
    
    if omega != None:
        w = omega

    if isinstance(vetor_x, int): # Se não é fornecido um vetor de aproximação inicial para x, ele é iniciado aqui com todos os elementos iguais a zero
        vetor_x = np.zeros(m)

    vetor_aux = np.copy(vetor_x) # Vetor auxiliar igual ao vetor x que representa a iteração k+1

    if criterio_linhas(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério das linhas foi satisfeito: O sistema converge.")
    else:
        print("ATENÇÃO: O critério das linhas não foi satisfeito.")

    if criterio_sassenfeld(matriz_A): # Verifica se o critério das linhas é satisfeito
        print("O critério de Sassenfeld foi satisfeito.")
    else:
        print("ATENÇÃO: O critério de Sassenfeld não foi satisfeito.")

    for k in range(N): # Executa todas as iterações especificadas na chamada da função
        for i in range(n):
            for j in range(m):
                if i != j: # Pula os elementos da diagonal principal pois eles não são utilizados
                    if i > j:
                        aux += matriz_A[i,j] * vetor_aux[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores que já foram atualizados (de 0 a i) que ficam no vetor auxiliar
                    else:
                        aux += matriz_A[i,j] * vetor_x[j] # Acumula as subtrações dos valores de Aij*xj utilizando os valores ainda não foram atualizados (de i+1 a n) que ficam no vetor x

            vetor_aux[i] = (1-w)*vetor_x[i] + (w*(vetor_b[i] - aux))/matriz_A[i,i] # Executa a iteração do método com (1-w)*xi(bi-Aij*xj)/Aii
            aux = 0 # Zera a contagem do somatório dos valores novamente

        termos_encontrados = 0
        for i in range(n): #Verifica elemento por elemento de vetor_x e do vetor_aux
            if vetor_x[i] != 0:
                if np.abs(vetor_aux[i] - vetor_x[i])/np.abs(vetor_x[i]) <= p: # Condição de parada caso a variação relativa de uma iteração para outra seja menor que a tolerância p 
                    termos_encontrados += 1
            else:
                if np.abs(vetor_aux[i] - vetor_x[i]) <= p*np.abs(vetor_x[i]):
                    termos_encontrados += 1
                    
        if(termos_encontrados == n):
                print("(SOR) Resultado encontrado com o critério de parada, na iteração:", k)
                return vetor_x

        vetor_x = np.copy(vetor_aux) # Atualiza o vetor_x (k) com os valores do vetor_aux (k+1)

    print("(SOR) Resultado encontrado com o número máximo de iterações.")
    return vetor_x # Retorna o vetor solução

In [111]:
a = np.array([[2.,1.,0.],[1.,2.,-1.],[0.,-1.,2.]])

In [112]:
b = np.array([[3.],[0.],[1.]])

In [113]:
x = np.array([[0.],[0.],[0.]])

In [114]:
print(jacobi(a, b, x))
print(seidel(a, b, x))
print(sor(a, b, x))

ATENÇÃO: O critério das linhas não foi satisfeito.
(Jacobi) Resultado encontrado com o critério de parada, na iteração: 99
[[ 2.0000000e+00]
 [-1.0000000e+00]
 [ 8.8817842e-16]]
ATENÇÃO: O critério das linhas não foi satisfeito.
Maior betão: 2.25
ATENÇÃO: O critério de Sassenfeld não foi satisfeito.
(Seidel) Resultado encontrado com o critério de parada, na iteração: 52
[[ 2.]
 [-1.]
 [ 0.]]
omega: 1.17157287525381
ATENÇÃO: O critério das linhas não foi satisfeito.
Maior betão: 2.25
ATENÇÃO: O critério de Sassenfeld não foi satisfeito.
(SOR) Resultado encontrado com o critério de parada, na iteração: 424
[[ 2.]
 [-1.]
 [ 0.]]


---
### 5) Implemente um algoritmo que leia uma matriz A (de uma planilha excel) e:

In [115]:
def ler_matriz_A(file_matrix, sheet_matriz):
    df_matriz = pd.read_excel(r"dados/"+file_matrix+".xlsx", header=None, sheet_name=sheet_matriz) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    return matriz

In [116]:
matriz = ler_matriz_A("prova_ALC_testes", "matriz_A(2)")

**a)** Calcule a transposta de A;

In [117]:
def calcula_transposta(matriz_A):
    if not verifica_quadrada(matriz_A):
        print("A matriz não é quadrada.")
        return None

    n = len(matriz_A)
    matriz_At = np.zeros((n,n), dtype='f8')

    for i in range(n):
        for j in range(n):
            if i == j:
                matriz_At[i,j] = matriz_A[i,j]
            else:
                matriz_At[i,j] = matriz_A[j,i]
    
    return matriz_At
    

In [118]:
Matrix(calcula_transposta(matriz))

⎡91452.0  55679.0  74196.0   50369.0  65933.0  58983.0  57282.0  86619.0   776
⎢                                                                             
⎢55679.0  65707.0  74216.0   37748.0  57071.0  53220.0  58214.0  72412.0   576
⎢                                                                             
⎢74196.0  74216.0  106855.0  53876.0  66014.0  75409.0  71392.0  100305.0  776
⎢                                                                             
⎢50369.0  37748.0  53876.0   48748.0  41012.0  44969.0  38317.0  58496.0   478
⎢                                                                             
⎢65933.0  57071.0  66014.0   41012.0  79341.0  54990.0  48071.0  73169.0   686
⎢                                                                             
⎢58983.0  53220.0  75409.0   44969.0  54990.0  81033.0  55404.0  71312.0   695
⎢                                                                             
⎢57282.0  58214.0  71392.0   38317.0  48071.0  55404

**b)** Calcule o determinante de A;

In [119]:
def eliminacao_gauss(matriz_A):
    n = len(matriz_A)
    Am = np.array(matriz_A).astype("float64")

    for i in range(n):
        for j in range(i+1, n):
            if Am[i,i] == 0:
                Am[i,i] = 1e-18

            x = Am[j,i]/Am[i,i] # x é o multiplicador necessário para executar a eliminação
            for k in range(n):
               Am[j,k] -= Am[i,k] * x # aplica o multiplicador x para zerar os elementos abaixo da diagonal principal

    return calcula_determinante(Am, n) # retorna o vetor resultado

def calcula_determinante(matriz_A, n):
    det = 1
    for i in range(n):
        det *= matriz_A[i,i]
    return det


In [120]:
eliminacao_gauss(matriz)

1.8092256407162663e+97

**c)** Calcule a matriz adjunta de A;

In [121]:
def calcula_adjunta(matriz_A):
    if verifica_inversa(matriz_A):
        matriz_Adj = np.linalg.inv(matriz_A) * eliminacao_gauss(matriz_A)
        return matriz_Adj

In [122]:
print(calcula_adjunta(matriz))

A matriz é quadrada
O determinante da matriz é: 1.8092256407074624e+97
[[ 2.18976648e+96  3.29994217e+96  5.58389985e+95  2.92783527e+95
  -2.03534984e+96 -2.36899618e+96 -5.50695114e+96  5.56505936e+95
  -3.82070029e+95 -5.35550547e+96  1.49829376e+96 -1.61176699e+96
   1.73225255e+96  1.54159730e+96  3.86518043e+95 -1.03851399e+96
  -1.03318594e+96  3.84136131e+95  2.15896487e+96 -5.31253148e+95
   3.99371498e+96  4.93413144e+95  6.98444992e+94 -3.03110404e+95
  -8.49040544e+95]
 [ 3.29994217e+96  4.98290950e+96  8.39954897e+95  4.43369933e+95
  -3.07182723e+96 -3.57137721e+96 -8.30586076e+96  8.42264733e+95
  -5.74222763e+95 -8.07654383e+96  2.26034329e+96 -2.43160828e+96
   2.61440020e+96  2.32211712e+96  5.78620430e+95 -1.56446934e+96
  -1.55871781e+96  5.80478295e+95  3.25585364e+96 -8.03621022e+95
   6.02497461e+96  7.40936689e+95  1.06077010e+95 -4.55738134e+95
  -1.28356107e+96]
 [ 5.58389985e+95  8.39954897e+95  1.46970788e+95  7.58187475e+94
  -5.17157626e+95 -6.05386182e+95

**d)** Calcule o traço de A;

In [123]:
def traco(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    n, traco = min((linhas, colunas)), 0

    for i in range(n):
        traco += matriz_A[i,i]
    return traco


In [124]:
print("Traço de A:", traco(matriz))

Traço de A: 2024668.0


**e)** Avalie se A é simétrica;

In [125]:
def verifica_simetria(matriz_A):
    if verifica_quadrada(matriz_A):
        n = len(matriz_A)
        for i in range(n):
            for j in range(n):
                if (i != j) and (matriz_A[i,j] != matriz_A[j,i]):
                    print("A matriz não é simétrica")
                    return False
        print("A matriz é simétrica")
        return True
    print("A matriz não é quadrada")
    return False

In [126]:
verifica_simetria(matriz)

A matriz é simétrica


True

**f)** Avalie se A é ortogonal;

In [127]:
def verifica_ortogonal(matriz_A):
    if not verifica_quadrada:
        print("A matriz não é quadrada.")
        return False
    
    if not verifica_inversa:
        print("A matriz é singular.")
        return False
    
    n = len(matriz_A)
    matriz_At = calcula_transposta(matriz_A)
    matriz_Id = np.identity(n)

    matriz_Resultado = np.matmul(matriz_A, matriz_At)

    if (matriz_Resultado-matriz_Id).all() <= 1e-10:
        print("A matriz é ortogonal.")
        return True
    else:
        print("A matriz não é ortogonal.")
        return False

In [128]:
verifica_ortogonal(matriz)

A matriz não é ortogonal.


False

**g)** Calcule a norma de linha, norma de Frobenius e norma coluna de A.

In [129]:
def soma_linha(matriz_A): # norma-infinita matricial
    n, max, x = len(matriz_A), 0, 0

    for i in range(n):
        for j in range(n):
            x += math.fabs(matriz_A[i,j])
        
        if x > max:
            max = x

    return max

In [130]:
soma_linha(matriz)

37848742.0

In [131]:
def frobenius(matriz_A): # norma-2 matricial
    n, x = len(matriz_A), 0

    for i in range(n):
        for j in range(n):
            x += math.pow(math.fabs(matriz_A[i,j]), 2)

    return x ** (1/2)

In [132]:
frobenius(matriz)

1544623.5421512907

In [133]:
def soma_coluna(matriz_A): # norma-1 matricial
    n, max, x = len(matriz_A), 0, 0

    for j in range(n):
        for i in range(n):
            x += math.fabs(matriz_A[i,j])
        
        if x > max:
            max = x
    
    return max

In [134]:
soma_coluna(matriz)

37848742.0

**h)** Avalie se A é positiva definida.

In [135]:
def checa_positiva_definida(matriz_A, debug=False):
    if not verifica_inversa or not verifica_quadrada or not verifica_simetria:
        return False

    n = len(matriz_A) # n é a ordem da matriz a
    matriz_R = np.copy(np.triu(matriz_A)) # matriz_r é uma cópia triangular superior da matriz A

    for i in range(n):
            for k in range(i): # Não é executado quando i = 0
                matriz_R[i,i] = matriz_R[i,i] - math.pow(matriz_R[k,i],2) # Define os termos da diagonal principal
            if matriz_R[i,i] <= 0: # Se algum elemento da diagonal principal for menor ou igual a zero, por definição, a matriz não é positiva definida
                if debug:
                    print("Erro: A matriz não é positiva definida.")
                return False
            matriz_R[i,i] = math.sqrt(matriz_R[i,i]) 
            for j in range(i+1,n): # Não é executado quando i = n
                for k in range(i): # Não é executado quando i = 0
                    matriz_R[i,j] = matriz_R[i,j] - (matriz_R[k,i] * matriz_R[k,j])
                matriz_R[i,j] = matriz_R[i,j]/matriz_R[i,i]
    return True

In [136]:
checa_positiva_definida(matriz)

True

---
### 6) Construa um algoritmo que receba dois vetores x e y e uma matriz A positiva definida (de uma planilha excel). Calcule:

In [137]:
def ler_matriz_A_vetor_x_vetor_y(file_name, file_matrix, file_array_x, file_array_y):
    df_matriz = pd.read_excel(r"dados/"+file_name+".xlsx", header=None, sheet_name=file_matrix) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor_x = pd.read_excel(r"dados/"+file_name+".xlsx", header=None, sheet_name=file_array_x) # Le o arquivo do excel que contém o vetor x sem cabeçalho
    df_vetor_y = pd.read_excel(r"dados/"+file_name+".xlsx", header=None, sheet_name=file_array_y) # Le o arquivo do excel que contém o vetor y sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor_x = np.ravel(np.array(df_vetor_x, dtype='f8')) # Converte o vetor b em um array da numpy
    vetor_y = np.ravel(np.array(df_vetor_y, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor_x, vetor_y


In [138]:
matriz, vetor_x, vetor_y = ler_matriz_A_vetor_x_vetor_y("prova_ALC_testes", "matriz_A(2)", "vetor_b(1)", "vetor_b(1)")

**a)** Norma 1, norma 2 e norma infinito de x e y.

In [139]:
def manhattan(vetor): # norma-1 vetorial
    n, x = len(vetor), 0

    for i in range(n):
        x += np.fabs(vetor[i])
    return x

In [140]:
print(manhattan(vetor_x))
print(manhattan(vetor_y))

26072.0
26072.0


In [141]:
def euclidiana(vetor): # norma-2 vetorial
    n, x = len(vetor), 0

    for i in range(n):
        x += np.fabs(vetor[i]) ** 2

    return x ** (1/2)

In [142]:
print(euclidiana(vetor_x))
print(euclidiana(vetor_y))

5264.014627639251
5264.014627639251


In [143]:
def infinita(vetor): # norma-infinita vetorial
    n, max = len(vetor), vetor[0]
    
    for i in range(1, n):
        if np.fabs(vetor[i]) > max:
            max = np.fabs(vetor[i])

    return max

In [144]:
print(infinita(vetor_x))
print(infinita(vetor_y))

1291.0
1291.0


**b)** Ângulo entre x e y;

In [145]:
def produto_interno(vetor_x, vetor_y):
    n, p_interno = len(vetor_x), 0

    for i in range(n):
        p_interno += vetor_x[i] * vetor_y[i]
    
    return p_interno

def angulo_vetores(vetor_x, vetor_y):
    n = len(vetor_x)

    p_interno = produto_interno(vetor_x, vetor_y)
    norma_x = euclidiana(vetor_x)
    norma_y = euclidiana(vetor_y)
    p_normas = norma_x * norma_y

    div = p_interno/p_normas

    return np.arccos(div)

In [146]:
angulo_vetores(vetor_x, vetor_y)

0.0

**c)** Norma de x e y induzida por A;

**d)** Produto interno de x e y.

In [147]:
def produto_interno(vetor_x, vetor_y):
    n, p_interno = len(vetor_x), 0

    for i in range(n):
        p_interno += vetor_x[i] * vetor_y[i]
    
    return p_interno

In [148]:
produto_interno(vetor_x, vetor_y)

27709850.0

---
### 7) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel). Encontre a fatoração LU de A.

In [149]:
matriz, vetor = ler_matriz_A_vetor_b("prova_ALC_testes", "matriz_A(1)", "vetor_b(1)")

In [150]:
fatoracaoLU(matriz,vetor)

array([ 218.15690039, -134.91978861, -228.57366314,  196.17807308,
        -23.10822144,  120.75613378,   76.57658357,   32.56339464,
        112.28655838, -311.76230605,    9.73618607,  179.86106078,
         37.55533057,   -5.28610392,   20.3176418 ,  120.6584339 ,
        184.2906244 ,   34.16170845, -178.63243142,  -42.98537213,
       -108.87178633,  -70.23227857,  168.6609164 , -259.13412099,
       -164.019284  ])

---
### 8) Construa um algoritmo que receba uma matriz A positiva (de uma planilha excel). Calcule:

In [151]:
matriz = ler_matriz_A("prova_ALC_testes", "matriz_A(2)")

**a)** O número condição;

**b)** Retorne uma mensagem ao usuário sobre sua interpretação.

---
### 9) Construa um algoritmo que receba três matrizes: U, sigma e V (de uma planilha excel) e:

In [152]:
def ler_matrizes_USV(file_excel):
    df_matriz_U = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name='matriz_U') # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_matriz_S = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name='matriz_S') # Le o arquivo do excel que contém o vetor x sem cabeçalho
    df_matriz_V = pd.read_excel(r"dados/"+file_excel+".xlsx", header=None, sheet_name='matriz_V') # Le o arquivo do excel que contém o vetor y sem cabeçalho
    matriz_U = np.array(df_matriz_U, dtype='f8') # Converte a matriz A em um array da numpy
    matriz_S = np.array(df_matriz_S, dtype='f8') # Converte a matriz A em um array da numpy
    matriz_Vt = np.array(df_matriz_V, dtype='f8') # Converte a matriz A em um array da numpy
    return matriz_U, matriz_S, matriz_Vt

matriz_U, matriz_S, matriz_Vt = ler_matrizes_USV("prova_ALC_testes")

**a)** Avalie se o produto das três matrizes representa uma decomposição SVD de alguma matriz X;

**b)** Caso a resposta à pergunta anterior seja negativa, retorne ao usuário uma explicação justificando esse fato.

In [179]:
# U e V ortogonais e S não-quadrada e diagonal
def verifica_diagonal(matriz):
    n, m = np.shape(matriz)

    for i in range(n):
        for j in range(m):
            if i != j and matriz[i,j] != 0:
                print("A matriz não é diagonal.")
                return False
    print("A matriz é diagonal.")
    return True

def verifica_SVD(matriz_U, matriz_S, matriz_Vt):
    print("Para as matrizes USVt serem a decomposição SVD de uma matriz X:\n-> U e Vt devem ser ortogonais.\n-> S deve ser diagonal não-quadrada.\n")
    print("Verificando se U é ortogonal...")
    if verifica_ortogonal(matriz_U):
        print("\nVerificando se Vt é ortogonal...")
        if verifica_ortogonal(matriz_Vt):
            print("\nVerificando se S é diagonal e não-quadrada...")
            if verifica_diagonal(matriz_S):
                nS, mS = np.shape(matriz_S)
                if nS != mS:
                    print("A matriz é não-quadrada.")
                    print("")
                    mU = len(matriz_U)
                    nVt = len(matriz_Vt)

                    if mU == nS and mS == nVt:
                        print("As matrizes são a decomposição SVD da matriz X abaixo:")
                        matriz_X = np.matmul(np.matmul(matriz_U, matriz_S), matriz_Vt)
                        print(matriz_X)

                        return True
                    else:
                        print("As matrizes não têm dimensões compatíveis para executar um produto de matrizes.")
                        return False
                else:
                    print("A matriz S é quadrada.")
                    return False
            else:
                print("A matriz S não é diagonal.")
                return False
        else:
            print("A matriz Vt não é ortogonal.")
            return False
    else:
        print("A matriz U não é ortogonal.")
        return False

In [180]:
verifica_SVD(matriz_U, matriz_S, matriz_Vt)

Para as matrizes USVt serem a decomposição SVD de uma matriz X:
-> U e Vt devem ser ortogonais.
-> S deve ser diagonal não-quadrada.

Verificando se U é ortogonal...
A matriz é ortogonal.

Verificando se Vt é ortogonal...
A matriz é ortogonal.

Verificando se S é diagonal e não-quadrada...
A matriz é diagonal.
A matriz é não-quadrada.

As matrizes são a decomposição SVD da matriz X abaixo:
[[2. 1.]
 [1. 2.]
 [0. 0.]]


True

---
### 10) Construa um algoritmo que leia uma planilha em Excel com 4 colunas e calcule a estatística de Benford (para 1 dígito) para cada uma das colunas.

In [155]:
benford(ler_dados_benford("dados_benford"))

Porcentagem de ocorrência de "1" em cada coluna: 10.6692% | 11.1430% | 11.8036% | 10.9420% | 
Porcentagem de ocorrência de "2" em cada coluna: 11.1574% | 11.0281% | 10.8271% | 11.3297% | 
Porcentagem de ocorrência de "3" em cada coluna: 11.3297% | 10.8415% | 11.3441% | 11.2005% | 
Porcentagem de ocorrência de "4" em cada coluna: 11.3584% | 10.5112% | 10.3676% | 10.7553% | 
Porcentagem de ocorrência de "5" em cada coluna: 10.6117% | 11.1430% | 11.2866% | 11.0999% | 
Porcentagem de ocorrência de "6" em cada coluna: 11.2435% | 11.3584% | 10.8845% | 11.0425% | 
Porcentagem de ocorrência de "7" em cada coluna: 11.8323% | 11.1861% | 11.4733% | 10.7266% | 
Porcentagem de ocorrência de "8" em cada coluna: 11.0999% | 11.1717% | 11.4159% | 11.6743% | 
Porcentagem de ocorrência de "9" em cada coluna: 10.6979% | 11.6169% | 10.5974% | 11.2005% | 
